# 特徴量選択
- 利用している特徴量の有用性を可視化する
- 新たな特徴量の作成を試みる

In [1]:
from backend.module.preparing import TableMerger
from backend.module.simulation import RankPredictor
from backend.module.repository import (
    Peds,
    Result,
    Race,
    HorseProfile,
    RaceCard,
    OriginalRaceCard,
    Refund,
)
from backend.module.handler import (
    PredictionBrancher,
    PredictionExtractor,
    RefundCalculator,
)
from backend.module.record import PredictionWriter

from backend.environment.mapping import Mapping
from backend.environment.columns import Columns

2022-11-12 03:01:01,738 INFO sqlalchemy.engine.Engine select pg_catalog.version()
2022-11-12 03:01:01,739 INFO sqlalchemy.engine.Engine [raw sql] {}
2022-11-12 03:01:01,740 INFO sqlalchemy.engine.Engine select current_schema()
2022-11-12 03:01:01,740 INFO sqlalchemy.engine.Engine [raw sql] {}
2022-11-12 03:01:01,742 INFO sqlalchemy.engine.Engine show standard_conforming_strings
2022-11-12 03:01:01,743 INFO sqlalchemy.engine.Engine [raw sql] {}
2022-11-12 03:01:01,746 INFO sqlalchemy.engine.Engine select relname from pg_class c join pg_namespace n on n.oid=c.relnamespace where pg_catalog.pg_table_is_visible(c.oid) and relname=%(name)s
2022-11-12 03:01:01,747 INFO sqlalchemy.engine.Engine [generated in 0.00103s] {'name': 'SELECT race_date_mapping.race_id, race_date_mapping.date \nFROM race_date_mapping'}
2022-11-12 03:01:01,750 INFO sqlalchemy.engine.Engine SELECT race_date_mapping.race_id, race_date_mapping.date 
FROM race_date_mapping
2022-11-12 03:01:01,751 INFO sqlalchemy.engine.Engi

In [2]:
import numpy as np
import pandas as pd
import requests
from bs4 import BeautifulSoup
from datetime import date

In [3]:
train_data = pd.read_pickle('temp.pickle')
train_data = train_data[train_data['date'] < date(2022, 9, 4)]
X = train_data.drop(Columns.DROP_COLUMNS+['rank', 'popular', 'odds'], axis=1)
y = train_data['rank']

In [12]:
train_data.columns

Index(['jockey_id', 'bracket_number', 'horse_number', 'weight_carry',
       'popular', 'age', 'weight_horse', 'diff_weight_horse', 'date',
       'course_len', 'nth_race', 'nth_day', 'nth_time', 'venue', 'month_sin',
       'month_cos', 'n_horses', 'breeder_id', 'owner_id', 'trainer_id',
       'avg_order_for_allR_by_breeder_id',
       'avg_time_idx_for_allR_by_breeder_id',
       'avg_prize_for_allR_by_breeder_id', 'avg_order_for_allR_by_owner_id',
       'avg_time_idx_for_allR_by_owner_id', 'avg_prize_for_allR_by_owner_id',
       'avg_order_for_allR_by_trainer_id',
       'avg_time_idx_for_allR_by_trainer_id',
       'avg_prize_for_allR_by_trainer_id', 'birthday',
       'avg_first_corner_for_allR_by_horse_id',
       'avg_final_corner_for_allR_by_horse_id',
       'avg_first_to_rank_for_allR_by_horse_id',
       'avg_first_to_final_for_allR_by_horse_id',
       'avg_final_to_rank_for_allR_by_horse_id',
       'avg_last3F_for_allR_by_horse_id', 'avg_diff_time_for_allR_by_horse_id'

## 1-1 weather

In [16]:
train_data[['weather_晴', 'weather_雨', 'weather_小雨', 'weather_小雪', 'weather_雪', 'rank']].head(60)

weather_晴  weather_雨  weather_小雨  weather_小雪  \
race_id      horse_id                                                   
201606010101 2013105621          1          0           0           0   
             2013104555          1          0           0           0   
             2013104573          1          0           0           0   
             2013102283          1          0           0           0   
             2013100107          1          0           0           0   
             2013100103          1          0           0           0   
             2013103101          1          0           0           0   
             2013104668          1          0           0           0   
             2013103416          1          0           0           0   
             2013106151          1          0           0           0   
             2013102845          1          0           0           0   
             2013100513          1          0           0           0   
             2013102932          1          0           0           0   
             2013102277          1          0           0           0   
             2013104382          1          0           0           0   
             2013103702          1          0           0           0   
201606010102 2013100549          1          0           0           0   
             2013101182          1          0           0           0   
             2013102919          1          0           0           0   
             2013109101          1          0           0           0   
             2013100001          1          0           0           0   
             2013100653          1          0           0           0   
             2013101939          1          0           0           0   
             2013105713          1          0           0           0   
             2013101957          1          0           0           0   
             2013103597          1          0           0           0   
             2013105676          1          0           0           0   
             2013104107          1          0           0           0   
             2013106344          1          0           0           0   
             2013105125          1          0           0           0   
             2013101293          1          0           0           0   
             2013101914          1          0           0           0   
201606010103 2012105911          1          0           0           0   
             2012102781          1          0           0           0   
             2011104104          1          0           0           0   
             2011105378          1          0           0           0   
             2012101907          1          0           0           0   
             2011104597          1          0           0           0   
             2011104171          1          0           0           0   
             2011101687          1          0           0           0   
             2011104863          1          0           0           0   
             2012105673          1          0           0           0   
             2011102712          1          0           0           0   
             2011102141          1          0           0           0   
             2012100798          1          0           0           0   
             2012102008          1          0           0           0   
             2011105789          1          0           0           0   
             2011101394          1          0           0           0   
201606010104 2012101566          1          0           0           0   
             2011104070          1          0           0           0   
             2011103677          1          0           0           0   
             2012104216          1          0           0           0   
             2011102816          1          0           0           0   
          